# asal code

In [2]:
#date : 1.4.24

import cv2
import tkinter as tk
import os
import numpy as np
import time

### idhi github la kelli eskochina already pre-trained model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)


cam = cv2.VideoCapture(0)


cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
current_dress_index = 0
user_picture_path = r'.\assets'

################################ kindha code gpt la kelli eskochindhi ###################################

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        #print('Selected shirt image:', image_path) #idhi anvasaram ga print chesthandhi errors appudu enable cheskundham

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

def process_dress_image(img): ## dheeni baadha endho kuda ardham aiethaledh, okka transparency osthey aiepothadhi ga saava dobbhuthandhi
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            # Split channels
            b, g, r = cv2.split(img)
            
            # Create alpha channel (fully opaque)
            alpha = np.ones_like(b) * 255
            
            # Merge channels to form RGBA image
            rgba_img = cv2.merge((b, g, r, alpha))
            
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

    
################################ paina code gpt la kelli eskochindhi but waste ###################################

def apply_shirt(frame, dress_index, detections):
    global dress_images
    if dress_images and dress_index < len(dress_images):
        confidence_threshold = 0.5  
        detected_faces = []
        
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            
            if confidence > confidence_threshold:
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                detected_faces.append((x, y, w, h))

        if detected_faces:
            dress_img = dress_images[dress_index]
            aspect_ratio = dress_img.shape[1] / dress_img.shape[0]

            for (x, y, w, h) in detected_faces:
                y_shirt = y + int(0.8 * h)
                h_shirt = int(0.5 * h)
                x_offset = int(0.5 * w)
                x_shirt = max(x - x_offset, 0)
                w_shirt = min(w + 2 * x_offset, frame.shape[1] - x_shirt)
                new_h_shirt = int(w_shirt / aspect_ratio)

                if new_h_shirt > 0:
                    resized_shirt = cv2.resize(dress_img, (w_shirt, new_h_shirt), interpolation=cv2.INTER_LINEAR)

                    if resized_shirt.shape[2] == 4:
                        resized_shirt = resized_shirt[:, :, :3]

                    frame[y_shirt:y_shirt + new_h_shirt, x_shirt:x_shirt + w_shirt] = resized_shirt[
                                                                                        :min(new_h_shirt,
                                                                                             frame.shape[0] - y_shirt),
                                                                                        :min(w_shirt,
                                                                                             frame.shape[1] - x_shirt)]
                    
##########################   ee paina code gpt improvise chesindhi changes em cheyyodhu malla pisukuthadhi   ##########################
                    
def process_and_save_output(frame, dress_index, detections):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        apply_shirt(frame_copy, dress_index, detections)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

def main():
    select_dress_images()
    global current_dress_index

    start_time = time.time()
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        apply_shirt(frame, current_dress_index, detections)
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        elif key == 13:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
        
        if time.time() - start_time >= 1:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
            start_time = time.time()

    cv2.destroyAllWindows()
    cam.release()

main()


Error loading image: .\assets\shirt11.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png


# idhi endhuko kuda telvadhi 

## gpt ichina background removal codes but dheniki pani raadh

In [15]:
#for removing the background of a picture 
import cv2
import numpy as np

# Read image
shirts = []
for i in range(7):
    im = f'.\\assets\\shirt{i}.jpg'
    shirts.append(im)
    img = cv2.imread(im)
    hh, ww = img.shape[:2]

    # threshold on white
    # Define lower and uppper limits
    lower = np.array([200, 200, 200])
    upper = np.array([255, 255, 255])

    # Create mask to only select white
    thresh = cv2.inRange(img, lower, upper)

    # apply morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # invert morph image
    mask = 255 - morph

    # apply mask to image
    result = cv2.bitwise_and(img, img, mask=mask)

    # save results for each shirt
    cv2.imwrite(fr'.\assets\shirt{i}.jpg', result)

# save results
    '''
    cv2.imwrite('pills_thresh.jpg', thresh)
    cv2.imwrite('pills_morph.jpg', morph)
    cv2.imwrite('pills_mask.jpg', mask)'''

    # cv2.imwrite(im, result)

    # cv2.imshow('thresh', thresh)
    # cv2.imshow('morph', morph)
    # cv2.imshow('mask', mask)
    # cv2.imshow('result', result)

    # cv2.imshow(f'.\assets\shirt{i}', result)

cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
#to make the image tranparent
import cv2
import numpy as np

im = r'.\assets\shirt1.jpg'
mi = r'image_bgra.png'

# read the image
image_bgr = cv2.imread(im)
# get the image dimensions (height, width and channels)
h, w, c = image_bgr.shape
# append Alpha channel -- required for BGRA (Blue, Green, Red, Alpha)
image_bgra = np.concatenate([image_bgr, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
# create a mask where white pixels ([255, 255, 255]) are True
white = np.all(image_bgr == [255, 255, 255], axis=-1)
# change the values of Alpha to 0 for all the white pixels
image_bgra[white, -1] = 0
# save the image

cv2.imwrite(mi, image_bgra)

In [ ]:
import cv2
import numpy as np
import os

def process_dress_image(img):
    try:
        # Check if the image has an alpha channel
        if img.shape[2] == 4:
            return img
        else:
            # Convert black pixels to transparent
            b, g, r = cv2.split(img)
            alpha = np.where((b == 0) & (g == 0) & (r == 0), 0, 255).astype(np.uint8)
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(8):  # Assuming you have 8 shirt images
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        print('Selected shirt image:', image_path)

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

# Example usage
dress_images = []
user_picture_path = './assets'
select_dress_images()
print('Number of shirt images processed:', len(dress_images))


In [ ]:
#no use black to transparent
# Import the library OpenCV 
import cv2 

# Import the image 
file_name = r".\assets\shirt3.png"

# Read the image 
src = cv2.imread(file_name, 1) 

# Convert image to image gray 
tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) 

# Applying thresholding technique 
_, alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY) 

# Using cv2.split() to split channels 
# of coloured image 
b, g, r = cv2.split(src) 

# Making list of Red, Green, Blue 
# Channels and alpha 
rgba = [b, g, r, alpha] 

# Using cv2.merge() to merge rgba 
# into a coloured/multi-channeled image 
dst = cv2.merge(rgba, 4) 

# Writing and saving to a new image 
cv2.imwrite("gfg_white.png", dst) 


In [41]:
import cv2
import numpy as np

# Load image
image = cv2.imread(r".\assets\shirt1.png")
cv2.imshow('Original Image', image)

# Create a mask initialized with zeros
mask = np.zeros(image.shape[:2], dtype=np.uint8)

# Initialize background and foreground models
bgdModel = np.zeros((1, 65), dtype=np.float64)
fgdModel = np.zeros((1, 65), dtype=np.float64)

# Define rectangle coordinates (x, y, width, height)
rect = (50, 30, 100, 200)

# Apply GrabCut algorithm
cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# Modify mask to create a binary mask (0: background, 1: foreground)
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

# Create a new blank image with the same dimensions as the input image
result = np.zeros_like(image)

# Copy the segmented foreground into the new image
result[mask2 == 1] = image[mask2 == 1]

# Display the result
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

# Load source image
source_img = cv2.imread('result_image.png')
source_img = cv2.resize(source_img, (400, 300))  # Resize for display (optional)
cv2.imshow('Source Image', source_img)

# Convert source image to grayscale
gray_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to create a binary mask
_, mask = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('Mask', mask)

# Apply the mask to the source image
result_img = cv2.bitwise_and(source_img, source_img, mask=mask)
cv2.imshow('Result without Transparent Background', result_img)

# Create a blank white image (background)
background = np.full_like(source_img, (255, 255, 255))

# Apply the inverse mask to the background
masked_background = cv2.bitwise_and(background, background, mask=cv2.bitwise_not(mask))

# Combine the masked background and the result image
result_with_mask = cv2.add(masked_background, result_img)
cv2.imshow('Result with Mask', result_with_mask)

# Add transparency (alpha channel) to the final result
rgba_result = cv2.cvtColor(result_with_mask, cv2.COLOR_BGR2BGRA)

# Set alpha channel values based on the mask
rgba_result[:, :, 3] = mask

cv2.imshow('Result with Transparent Background', rgba_result)
cv2.imwrite("a.png",rgba_result)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#from github code
import numpy as np
import cv2

img = cv2.imread('reseul_image.png')
overlay_t = cv2.imread('foreground_transparent.png',-1) # -1 loads with transparency

def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
	"""
	@brief      Overlays a transparant PNG onto another image using CV2
	
	@param      background_img    The background image
	@param      img_to_overlay_t  The transparent image to overlay (has alpha channel)
	@param      x                 x location to place the top-left corner of our overlay
	@param      y                 y location to place the top-left corner of our overlay
	@param      overlay_size      The size to scale our overlay to (tuple), no scaling if None
	
	@return     Background image with overlay on top
	"""
	
	bg_img = background_img.copy()
	
	if overlay_size is not None:
		img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

	# Extract the alpha mask of the RGBA image, convert to RGB 
	b,g,r,a = cv2.split(img_to_overlay_t)
	overlay_color = cv2.merge((b,g,r))
	
	# Apply some simple filtering to remove edge noise
	mask = cv2.medianBlur(a,5)

	h, w, _ = overlay_color.shape
	roi = bg_img[y:y+h, x:x+w]

	# Black-out the area behind the logo in our original ROI
	img1_bg = cv2.bitwise_and(roi.copy(),roi.copy(),mask = cv2.bitwise_not(mask))
	
	# Mask out the logo from the logo image.
	img2_fg = cv2.bitwise_and(overlay_color,overlay_color,mask = mask)

	# Update the original image with our new ROI
	bg_img[y:y+h, x:x+w] = cv2.add(img1_bg, img2_fg)

	return bg_img

cv2.imshow('image',overlay_transparent(img, overlay_t, 0, 0, (200,200)))
cv2.waitKey(0)

## i acutally need this below code for future
### this is the best code but runs very laggy and might make exceptions

In [ ]:
####### manam chesina code but chatgpt tkinter thoni upgradations ichindhi
#date : 29.3.24

import cv2
import tkinter as tk
from PIL import Image, ImageTk
import os
import numpy as np
import time


prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)


cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)


dress_images = []
user_picture_path = r'.\assets'
for i in range(6):
    image_path = os.path.join(user_picture_path, f'shirt{i}.jpg')
    print('Selected shirt images:', image_path)
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is not None:
        alpha_channel = np.ones((img.shape[0], img.shape[1], 1), dtype=np.uint8) * 255
        dress_images.append(np.concatenate((img, alpha_channel), axis=2))
    else:
        print(f'Error loading image: {image_path}')


current_dress_index = 0
start_time = time.time()


root = tk.Tk()
root.title("Virtual Try-On App")


video_label = tk.Label(root)
video_label.pack()


def update_video_feed():
    ret, frame = cam.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)
        photo = ImageTk.PhotoImage(image=image)
        video_label.config(image=photo)
        video_label.image = photo
    video_label.after(10, update_video_feed)


def apply_shirt(frame, dress_index):
    global dress_images
    if dress_images and dress_index < len(dress_images):
        detections = detect_faces(frame)
        if detections is not None:
            confidence_threshold = 0.5
            detected_faces = []
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > confidence_threshold:
                    box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                    (x, y, w, h) = box.astype(int)
                    detected_faces.append((x, y, w, h))

            if detected_faces:
                dress_img = dress_images[dress_index]
                aspect_ratio = dress_img.shape[1] / dress_img.shape[0]

                for (x, y, w, h) in detected_faces:
                    y_shirt = y + int(0.8 * h)
                    h_shirt = int(0.5 * h)
                    x_offset = int(0.5 * w)
                    x_shirt = max(x - x_offset, 0)
                    w_shirt = min(w + 2 * x_offset, frame.shape[1] - x_shirt)
                    new_h_shirt = int(w_shirt / aspect_ratio)

                    if new_h_shirt > 0:
                        resized_shirt = cv2.resize(dress_img, (w_shirt, new_h_shirt))
                        alpha_mask = resized_shirt[:, :, 3] / 255.0
                        alpha_mask = np.expand_dims(alpha_mask, axis=2)
                        resized_shirt = resized_shirt[:, :, :3]
                        frame_roi = frame[y_shirt:y_shirt + new_h_shirt, x_shirt:x_shirt + w_shirt]
                        frame_roi = cv2.addWeighted(frame_roi, 1.0, resized_shirt, 0.5, 0)
                        frame[y_shirt:y_shirt + new_h_shirt, x_shirt:x_shirt + w_shirt] = frame_roi

    return frame



def detect_faces(frame):
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    return net.forward()

def process_and_save_output(frame, dress_index):
    global start_time, current_dress_index
    output_folder = 'outputs'
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f'output{dress_index}.png')
    frame_copy = frame.copy()
    result_frame = apply_shirt(frame_copy, dress_index)
    cv2.imwrite(output_path, result_frame)
    print(f'Saved output image: {output_path}')
    current_dress_index = (current_dress_index + 1) % len(dress_images)
    start_time = time.time()


start_button = tk.Button(root, text="Start Try-On", command=lambda: process_and_save_output(cam.read()[1], current_dress_index))
start_button.pack()


def update_and_try_on():
    update_video_feed()
    if time.time() - start_time >= 5:
        process_and_save_output(cam.read()[1], current_dress_index)
    root.after(10, update_and_try_on)

update_and_try_on()
root.mainloop()


cv2.destroyAllWindows()
cam.release()
